# Installation

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

## Intall Pytorch2

In [1]:
!pip install torch torchvision torchaudio

%ls


In [9]:
%cd yolov5

/Users/sathya/Desktop/Learning/yolov5


/Users/sathya/opt/anaconda3/envs/YOLOV5/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [10]:
!pip3 install pickleshare


In [11]:
import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-392-g2d42ae1f Python-3.11.10 torch-2.2.2 CPU


Setup complete ✅ (8 CPUs, 16.0 GB RAM, 493.4/926.4 GB disk)


In [ ]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, trust_repo=True)  # or yolov5n - yolov5x6 or custom
im = 'yolov5/data/images/bus.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
results.print() 

In [11]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images

detect: weights=['yolov5s.pt'], source=data/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/Users/sathya/Desktop/Learning/Image': No such file or directory
YOLOv5 🚀 2024-11-19 Python-3.11.10 torch-2.2.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/2 /Users/sathya/Desktop/Learning/Image Recog/YoloV5/yolov5/data/images/bus.jpg: 640x480 4 persons, 1 bus, 339.6ms
image 2/2 /Users/sathya/Desktop/Learning/Image Recog/YoloV5/yolov5/data/images/zidane.jpg: 384x640 2 persons, 2 ties, 260.8ms
Speed: 0.9ms pre-process, 300.2ms infere

## Download COCO Dataset

In [12]:
# Download COCO val
torch.hub.download_url_to_file('https://github.com/ultralytics/assets/releases/download/v0.0.0/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

100%|████████████████████████████████████████| 780M/780M [01:27<00:00, 9.36MB/s]


### Optional validation using the detect.py and train.py scripts on the coco dataset

In [ ]:
!python detect.py --weights yolov5s.pt --source '/Users/sathya/Desktop/Learning/Image Recog/YoloV5/datasets/coco/images/val2017' --img 640

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache

### Generate the labels for the Mask at a random probability of 0.3 on all the existing images

In [4]:
import os
import numpy as np
import pandas as pd

# Define folder paths
input_folder = "/Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/labels/val2017"  # Folder containing the original text files
filtered_folder = "/Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/labels/labels_with_out_mask"  # Folder to save filtered files
removed_folder = "/Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/labels/mask"  # Folder to save removed rows

# Ensure the output folders exist
os.makedirs(filtered_folder, exist_ok=True)
os.makedirs(removed_folder, exist_ok=True)

# Probability threshold
threshold = 0.3

# Process each text file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):  # Process only .txt files
        input_file_path = os.path.join(input_folder, filename)
        
        # Read the text file into a DataFrame
        data = pd.read_csv(input_file_path, delimiter="\t", header=None, names=["Row"], dtype=str)

        # Generate random probabilities for each row
        data['Random_Probability'] = np.random.rand(len(data))

        # Filter rows based on the threshold
        filtered_data = data[data['Random_Probability'] >= threshold].drop(columns=['Random_Probability'])
        removed_data = data[data['Random_Probability'] < threshold].drop(columns=['Random_Probability'])

        # Save the filtered rows to the filtered folder
        filtered_file_path = os.path.join(filtered_folder, filename)
        filtered_data.to_csv(filtered_file_path, index=False, header=False, sep="\t")

        # Save the removed rows to the removed folder
        removed_file_path = os.path.join(removed_folder, filename)
        removed_data.to_csv(removed_file_path, index=False, header=False, sep="\t")

print("Processing complete. Filtered and removed files saved to respective folders.")


Processing complete. Filtered and removed files saved to respective folders.


### Based on the mask labels generated in the previous step, generate the respective mask images in PNG format. Ensure that each mask image has the same dimensions as the corresponding input original image. Store all the generated mask images in a designated folder for further use.

In [6]:
import os
from PIL import Image, ImageDraw

# Folder paths
image_folder = "/Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/images/val2017"  # Folder containing images
bbox_folder = "/Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/labels/mask"  # Folder containing bounding box txt files
output_folder = "/Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/images/mask"  # Folder to save generated mask images

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process each image in the image folder
for image_filename in os.listdir(image_folder):
    if image_filename.endswith((".jpg", ".png")):  # Process only image files
        # Get corresponding bounding box file
        base_name = os.path.splitext(image_filename)[0]
        bbox_file_path = os.path.join(bbox_folder, f"{base_name}.txt")

        # Skip if bounding box file doesn't exist
        if not os.path.exists(bbox_file_path):
            print(f"Bounding box file not found for {image_filename}. Skipping...")
            continue

        # Load image and get dimensions
        image_path = os.path.join(image_folder, image_filename)
        image = Image.open(image_path)
        image_width, image_height = image.size

        # Create a blank grayscale mask
        mask = Image.new("L", (image_width, image_height), 255)
        draw = ImageDraw.Draw(mask)

        # Read bounding box data from the text file
        with open(bbox_file_path, "r") as bbox_file:
            for line in bbox_file:
                values = line.strip().split()
                if len(values) != 5:
                    continue  # Skip invalid lines
                _, x_center, y_center, width, height = map(float, values)

                # Convert normalized coordinates to pixel coordinates
                box_width = width * image_width
                box_height = height * image_height
                x_min = (x_center * image_width) - (box_width / 2)
                y_min = (y_center * image_height) - (box_height / 2)
                x_max = x_min + box_width
                y_max = y_min + box_height

                # Draw rectangle on the mask (0 for inside the bounding box)
                draw.rectangle([x_min, y_min, x_max, y_max], fill=0)

        # Save the mask in the output folder as PNG
        mask_output_path = os.path.join(output_folder, f"{base_name}.png")
        mask.save(mask_output_path)
        print(f"Mask saved for {image_filename} at {mask_output_path}")

print("Processing complete.")


Mask saved for 000000182611.jpg at /Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/images/mask/000000182611.png
Mask saved for 000000335177.jpg at /Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/images/mask/000000335177.png
Mask saved for 000000278705.jpg at /Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/images/mask/000000278705.png
Mask saved for 000000463618.jpg at /Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/images/mask/000000463618.png
Mask saved for 000000568981.jpg at /Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/images/mask/000000568981.png
Mask saved for 000000092416.jpg at /Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/images/mask/000000092416.png
Mask saved for 000000173830.jpg at /Users/sathya/Desktop/Learning/Image_Recog/YoloV5/YoloV5_Mask/datasets/coco/images/mask/000000173830.png
Mask saved for 00000